In [1]:
from scipy.integrate import quad, trapz, fixed_quad
import theano
import theano.tensor as T
from theano.tests.unittest_tools import verify_grad
from theano.compile.ops import as_op

import numpy as np
import pymc3 as pm

import matplotlib.pyplot as plt

import astropy.units as u

from tqdm import tqdm

from gammapy.spectrum import CountsPredictor, CountsSpectrum

from utils import load_spectrum_observations, plot_spectra, Log10Parabola, integrate_spectrum, wstat_profile
from forward_fold_model import model_probability, ln_profile_likelihood

%matplotlib inline

In [2]:
class Integrate(theano.Op):
    def __init__(self, expr, var, lower, upper, *inputs):
        super().__init__()
        self._expr = expr
        self._var = var
        self._extra_vars = inputs
        self.lower = lower
        self.upper = upper
        self._func = theano.function(
            [var] + list(self._extra_vars),
            self._expr,
            on_unused_input='ignore'
        )
    
    def make_node(self, *inputs):
        assert len(self._extra_vars)  == len(inputs)
        return theano.Apply(self, list(inputs), [T.dscalar().type()])
    
    def perform(self, node, inputs, out):
        x = np.linspace(self.lower, self.upper, num=3)
        y = np.array([self._func(i , *inputs) for i in x])
        val = trapz(y, x)
#         print(val)
#         val = quad(self._func, self.lower, self.upper, args=tuple(inputs))[0]
        out[0][0] = np.array(val)
        
    def grad(self, inputs, grads):
        out, = grads
        grads = T.grad(self._expr, self._extra_vars)
        dargs = []
        for grad in grads:
            integrate = Integrate(grad, self._var, self.lower, self.upper, *self._extra_vars)
            darg = out * integrate(*inputs)
            dargs.append(darg)
            
        return dargs

In [3]:
def apply_range(*arr, fit_range, bins):
    idx = np.searchsorted(bins.to_value(u.TeV), fit_range.to_value(u.TeV))
    return [a[idx[0]:idx[1]] for a in arr]

In [23]:
def forward_fold_log_parabola_symbolic(amplitude, alpha, beta, observations, fit_range=None, efficiency=1):
    
    amplitude *= 1e-11
    
    predicted_signal_per_observation = []
    for observation in observations:
        obs_bins = observation.on_vector.energy.bins.to_value(u.TeV)


        aeff_bins = observation.aeff.energy
        e_reco_bins = observation.edisp.e_reco
        e_true_bins = observation.edisp.e_true

        lower =  e_true_bins.lo.to_value(u.TeV)
        upper = e_true_bins.hi.to_value(u.TeV)

        energy = T.dscalar('energy')
        amplitude_ = T.dscalar('amplitude_')
        alpha_ = T.dscalar('alpha_')
        beta_ = T.dscalar('beta_')

        func = amplitude_ * energy **(-alpha_ - beta_ * T.log10(energy))

        counts = []
        for a, b in zip(lower, upper):
            c = Integrate(func, energy, a, b, amplitude_, alpha_, beta_)(amplitude, alpha, beta)
            counts.append(c)

        counts = T.stack(counts)
        aeff = observation.aeff.data.data.to_value(u.cm**2).astype(np.float32)

        counts *= efficiency * aeff
        counts *= observation.livetime.to_value(u.s)
        edisp = observation.edisp.pdf_matrix
        edisp = edisp

        predicted_signal_per_observation.append(T.dot(counts, edisp))

    predicted_counts = T.sum(predicted_signal_per_observation, axis=0)
    if fit_range is not None:
        idx = np.searchsorted(obs_bins, fit_range.to_value(u.TeV))
        predicted_counts = predicted_counts[idx[0]:idx[1]]

    return predicted_counts

In [24]:
def forward_fold_log_parabola(amplitude, alpha, beta, observations, fit_range=None, efficiency=1):
    
    amplitude *= 1e-11
    
    predicted_signal_per_observation = []
    for observation in observations:
        obs_bins = observation.on_vector.energy.bins.to_value(u.TeV)


        aeff_bins = observation.aeff.energy
        e_reco_bins = observation.edisp.e_reco
        e_true_bins = observation.edisp.e_true

        lower =  e_true_bins.lo.to_value(u.TeV)
        upper = e_true_bins.hi.to_value(u.TeV)

        func = lambda energy: amplitude * energy **(-alpha - beta * np.log10(energy))

        counts = []
        for a, b in zip(lower, upper):
            x = np.linspace(a, b, num=3)
            y = np.array([func(i) for i in x])
            val = trapz(y, x)
            counts.append(val)

        counts = np.array(counts)
        aeff = observation.aeff.data.data.to_value(u.cm**2).astype(np.float32)

        counts *= efficiency * aeff
        counts *= observation.livetime.to_value(u.s)
        edisp = observation.edisp.pdf_matrix
        edisp = edisp

        predicted_signal_per_observation.append(np.dot(counts, edisp))

    predicted_counts = np.sum(predicted_signal_per_observation, axis=0)
    if fit_range is not None:
        idx = np.searchsorted(obs_bins, fit_range.to_value(u.TeV))
        predicted_counts = predicted_counts[idx[0]:idx[1]]

    return predicted_counts

In [25]:
observations, fit_range = load_spectrum_observations('fact', low_binning=False)
# observation = obs_list[1]
obs_alpha = observations[0].alpha[0]
energy_bins = observations[0].on_vector.energy.bins
print(len(energy_bins))
observations, fit_range # [obs.alpha for obs in observations]

73


([<gammapy.spectrum.observation.SpectrumObservation object at 0x7ff0198d31d0>],
 <Quantity [ 0.4, 30. ] TeV>)

In [26]:
forward_fold_log_parabola(4, 2.5, 0.4, observations, fit_range=fit_range)

array([ 3.18882116,  5.75584183,  9.85494299, 15.5439218 , 22.59697128,
       29.7498651 , 35.09259171, 38.84889437, 40.10574522, 38.54389662,
       36.19330187, 32.57614234, 27.83911836, 23.64408061, 19.67296688,
       15.9753555 , 13.20937235, 10.83104105,  8.65558815,  6.99811944,
        5.60007273,  4.38645758,  3.50388891,  2.76998654,  2.12690518,
        1.65219157,  1.25757336,  0.92742646,  0.69276249,  0.51152918,
        0.36767132,  0.26434699,  0.18044049,  0.11371709])

In [28]:
amplitude = T.dscalar('amplitude')
alpha = T.dscalar('alpha')
beta = T.dscalar('beta')

cf_fast = forward_fold_log_parabola_symbolic(amplitude, alpha, beta, observations, fit_range=fit_range)
counts_symbolic = cf_fast.eval({amplitude: 4.0, alpha: 2.5, beta: 0.4})
counts_symbolic, counts_symbolic.shape

(array([ 3.18882116,  5.75584183,  9.85494299, 15.5439218 , 22.59697128,
        29.7498651 , 35.09259171, 38.84889437, 40.10574522, 38.54389662,
        36.19330187, 32.57614234, 27.83911836, 23.64408061, 19.67296688,
        15.9753555 , 13.20937235, 10.83104105,  8.65558815,  6.99811944,
         5.60007273,  4.38645758,  3.50388891,  2.76998654,  2.12690518,
         1.65219157,  1.25757336,  0.92742646,  0.69276249,  0.51152918,
         0.36767132,  0.26434699,  0.18044049,  0.11371709]), (34,))

In [29]:
%timeit cf_fast.eval({amplitude: 4.0, alpha: 2.5, beta: 0.4})

22.6 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%timeit forward_fold_log_parabola(4, 2.5, 0.4, observations, fit_range=fit_range)

6.48 ms ± 651 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
